In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [ ]:
MAX_MEAT = 47
MAX_VEGGIE = 101
MAX_NOODLE = 268
BATCH_SIZE = 20
IMAGE_SIZE = (256, 256)

In [ ]:
df = pd.read_csv('./rechallenge_round/data.csv', header = 0, delimiter = ',')

In [ ]:
data_gen = ImageDataGenerator(rescale=1./255)

test_generator = data_gen.flow_from_dataframe(
    dataframe = df.loc[0 : 299],
    directory = './rechallenge_round/images',
    x_col = 'filename',
    y_col = [],
    shuffle = False,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
	class_mode= 'raw')

test_generator.reset()
model = load_model('model_best.h5')
predict = model.predict(
    test_generator,
    steps = len(test_generator),
    workers = 1,
    use_multiprocessing = False)

In [ ]:
result = []
test_generator.reset()
for (i, t) in enumerate(test_generator.labels) :
	file_name = str(i + 1) + '.jpg'
	meat, veggie, noodle = (predict[i][0] * MAX_MEAT, predict[i][1] * MAX_VEGGIE, predict[i][2] * MAX_NOODLE)
	result.append([file_name, meat, veggie, noodle])

df = pd.DataFrame(result)
df.head()

df.to_csv('./rechallenge_round/Result.csv', sep = ',', index = False, header = ['filename', 'meat', 'veggie', 'noodle'])